In [ ]:
! pip install -qU langchain-openai langchain langchain-community sentence-transformers unstructured[all-docs]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 61.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 4.8 MB/s eta 0:00:00
   ━━

In [ ]:
import docx
from unstructured.partition.docx import partition_docx
from pathlib import Path

In [ ]:
from typing import List
import docx
from unstructured.partition.docx import partition_docx
from pathlib import Path

def load_resumes(folder: str | Path, pattern: str = "*.docx") -> List[str]:
    """
    Load all .docx files from a folder into a dict:
      { "<filename>": "<plain_text>" }
    """
    folder = Path(folder)
    if not folder.exists():
        raise FileNotFoundError(f"Folder not found: {folder.resolve()}")

    file_list = []
    for file in sorted(folder.glob(pattern)):
        if file.is_file():
          file_list.append(file)

    return file_list

In [ ]:
file_list = load_resumes("/content/resume_db")
file_list

[PosixPath('/content/resume_db/Candidate 105.docx')]

In [ ]:
str(file_list[0].absolute())

'/content/resume_db/Candidate 105.docx'

# Partitioning the data

In [ ]:
docx_path = str(file_list[0].absolute())

elements = partition_docx(filename=docx_path,include_page_breaks=False,infer_table_structure=True)


In [ ]:
print(type(elements))

<class 'list'>


In [ ]:
print(type(elements[2]))

<class 'unstructured.documents.elements.ListItem'>


# Finding Resume Text

In [ ]:
resume_text = ""

for el in elements:
  if el.category.lower() != "header" and el.category.lower() != "footer":
    resume_text += el.text.lower() + "\n"


In [ ]:
import sys
sys.getsizeof(resume_text)

35620

# Call LLM to find MetaData like - experience_years_total,skills,certifications

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0.2,max_tokens=1500)

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
PROMPT_TEMPLATE = """
Act like an expert in extracting resume facts and respond in strict JSON :

given resume text : {resume_text}

Instructions:
1) Derive total exact experience found in resume text in years . Use decimals if possible.
2) Extract skills and Normalize skills to common names (map synonyms).
3) Include certifications (exact names) if present.
4) Extract Key Responsibilities of the candidate

Below are the valid key and value of the JSON Object -
"experience_years_total": number,
"skills": string[],
"certifications": string[],
"key_responsibilities": string[]

Don't include json keyword in the begining of output

"""

In [ ]:
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

In [ ]:
prompt

ChatPromptTemplate(input_variables=['resume_text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['resume_text'], input_types={}, partial_variables={}, template='\nAct like an expert in extracting resume facts and respond in strict JSON :\n\ngiven resume text : {resume_text}\n\nInstructions:\n1) Derive total experience in years from date ranges or explicit phrases. Use decimals.\n2) Extract top 5 skills and Normalize skills to common names (map synonyms).\n3) Include certifications (exact names) if present.\n4) Extract Key Responsibilities of the candidate\n\nBelow are the valid key and value of the JSON Object - \n"experience_years_total": number,\n"skills": string[], \n"certifications": string[],\n"key_responsibilities": string[]\n\nDon\'t include json keyword in the begining of output\n\n'), additional_kwargs={})])

In [ ]:
chain = prompt | llm | JsonOutputParser()
llm_response_json = chain.invoke({"resume_text":resume_text})
llm_response_json

{'experience_years_total': 14,
 'skills': ['SAP BW',
  'SAP BI',
  'ABAP',
  'BW/4HANA',
  'AMDP',
  'Native HANA',
  'BOBJ',
  'Power BI',
  'HANA Views',
  'DAX',
  'Power Automate',
  'BEx Analyzer',
  'BI Portal',
  'APDs',
  'QlikView',
  'S/4HANA',
  'Rebates Management',
  'Settlement Management',
  'PLM',
  'PM',
  'OTC',
  'CDS Views',
  'SDA',
  'Demantra',
  'ODP Framework',
  'ServiceNow',
  'JIRA',
  'Remedy',
  'Heat',
  'SAP BO Security',
  'Broadcasting',
  'SAP BOBJ WebI',
  'HANA Remediation',
  'Ticketing Tools'],
 'certifications': [],
 'key_responsibilities': ['Solution architect in analytics practice',
  'Solutioned webi to powerbi conversion project',
  'Created dataflows, datasets, reports, paginated reports',
  'Architected Power BI dashboard design',
  'Worked on BO 4.3 upgrade project',
  'Worked on BW/4HANA 1.0 green field implementation',
  'Worked on BW/4HANA 2.0 upgrade projects',
  'Worked extensively on SAP BI in various domains',
  'Worked with ABAP CD

In [ ]:
llm_response_json['key_responsibilities']

['Solution architect in analytics practice',
 'Solutioned webi to powerbi conversion project',
 'Created dataflows, datasets, reports, paginated reports',
 'Architected Power BI dashboard design',
 'Worked on BO 4.3 upgrade project',
 'Worked on BW/4HANA 1.0 green field implementation',
 'Worked on BW/4HANA 2.0 upgrade projects',
 'Worked extensively on SAP BI in various domains',
 'Worked with ABAP CDS views and native HANA modeling',
 'Provided key user training and proposed system improvements',
 'Involved in performance tuning techniques',
 'Worked on SAP BO security setup and broadcasting',
 'Involved in HANA upgradation project',
 'Worked through different ticketing tools',
 'Involved in presale for projects',
 'Part of pioneer green field implementation of S/4HANA 1610 system',
 'Enabled new settlement management document settings in S/4',
 'Designed PLM and PM BI architecture',
 'Worked on OTC push reports',
 'Leveraged CDS views functionality',
 'Created procedures for third-p

In [ ]:
key_responsibilities = [item.lower() for item in llm_response_json['key_responsibilities']]
key_responsibilities

['solution architect in analytics practice',
 'solutioned webi to powerbi conversion project',
 'created dataflows, datasets, reports, paginated reports',
 'architected power bi dashboard design',
 'worked on bo 4.3 upgrade project',
 'worked on bw/4hana 1.0 green field implementation',
 'worked on bw/4hana 2.0 upgrade projects',
 'worked extensively on sap bi in various domains',
 'worked with abap cds views and native hana modeling',
 'provided key user training and proposed system improvements',
 'involved in performance tuning techniques',
 'worked on sap bo security setup and broadcasting',
 'involved in hana upgradation project',
 'worked through different ticketing tools',
 'involved in presale for projects',
 'part of pioneer green field implementation of s/4hana 1610 system',
 'enabled new settlement management document settings in s/4',
 'designed plm and pm bi architecture',
 'worked on otc push reports',
 'leveraged cds views functionality',
 'created procedures for third-p

In [ ]:
from unstructured.chunking.title import chunk_by_title

# Apply Chunking - Find each chunks

In [ ]:

refined_elements = []

for el in elements:
  if el.category.lower() != "header" and el.category.lower() != "footer":
    refined_elements.append(el)

# Step 2: Apply chunking (chunk by title - assumes some sections have headers)
chunks = chunk_by_title(refined_elements)


# Optional: Convert chunks into raw text for further NLP
# This part also needs to handle the possibility that chunks might not be a list of lists
final_chunks_text = []

final_chunks_text = [chunk.text for chunk in chunks]

# Save to disk or further process
with open("chunked_output.txt", "w", encoding="utf-8") as f:
    for idx, text in enumerate(final_chunks_text):
        f.write(f"\n--- Chunk {idx + 1} ---\n{text}\n")

In [ ]:
final_chunks_text[2]

'Recently worked on BO 4.3 Upgrade project: Get the smoke test activities done on WEBI as well as CRYSTAL reports, Publications and WEBi job schedules across the landscape. Also, fixed a couple of issues faced with publication and transportation.\n\nWorked on traditional BW world from 2.x through BW 7.5 for different clients.\n\nWorked on BW/4HANA 1.0 Green field implementation and further worked on BW/4HANA 2.0 upgrade projects across the landscape. Recently, involved in 2.0 SP09 Migration.'

# Data cleaning and pre processing pipeline

In [ ]:
from unstructured.cleaners.core import bytes_string_to_string
from unstructured.cleaners.core import clean
from unstructured.cleaners.core import clean_bullets
from unstructured.cleaners.core import clean_dashes
from unstructured.cleaners.core import clean_non_ascii_chars
from unstructured.cleaners.core import clean_ordered_bullets
from unstructured.cleaners.core import clean_trailing_punctuation
from unstructured.cleaners.core import remove_punctuation

In [ ]:
def clean_chunks(all_chunks):
  cleaned_chunks = []
  for chunk in all_chunks:
    #refined_chunk = bytes_string_to_string(chunk, encoding="utf-8")
    #refined_chunk = clean(chunk, lowercase=True)
    refined_chunk = chunk.lower()
    refined_chunk = clean_bullets(refined_chunk)
    #refined_chunk = clean_dashes(refined_chunk)
    #refined_chunk = clean_non_ascii_chars(refined_chunk)
    #refined_chunk = clean_ordered_bullets(refined_chunk)
    refined_chunk = clean_trailing_punctuation(refined_chunk)
    #refined_chunk = remove_punctuation(refined_chunk)
    cleaned_chunks.append(refined_chunk)
  return cleaned_chunks



In [ ]:
cleaned_chunks = clean_chunks(final_chunks_text)

In [ ]:
for idx, chunk in enumerate(cleaned_chunks):
  print("chunk numer:: ", idx)
  print(chunk)

chunk numer::  0
professional summary

14 years of experience in sap bw/bi/abap, bw/4hana, amdp, native hana, bobj and powerbi.

working as a solution architect in analytics practice.

recently solutioned the webi to powerbi conversion project wherein converting all the webi reports into powerbi reports via hana views; basically, creating hana views following best practices code push down having all the calculations done at hana views then, expose them to powerbi
chunk numer::  1
created dataflows, datasets, reports, paginated reports consuming the sources from hana, sharepoint, and azure synapse. worked through the different dax formulas during the report design.

architected the powerbi expensive dashboard design; consuming hana expensive tables into views then dashboard at powerbi end. automated the flows and datasets through power automate
chunk numer::  2
recently worked on bo 4.3 upgrade project: get the smoke test activities done on webi as well as crystal reports, publications 

In [ ]:
cleaned_chunks[2]

'led the design of a predictive regression model that increased daily foot traffic by 30% in a retail chain.\n\nbuilt and deployed rag-based genai applications using langchain, openai, and hugging face transformers, enabling context-aware responses from large unstructured datasets.\n\nexperience\n\nlead engineer | epam systems | 02/2025 – present\n\ndeveloped ai-enabled contract and rate management platform for healthcare providers'

In [ ]:
len(cleaned_chunks)

41

In [ ]:
llm_response_json

{'experience_years_total': 14,
 'skills': ['SAP BW',
  'SAP BI',
  'ABAP',
  'BW/4HANA',
  'AMDP',
  'Native HANA',
  'BOBJ',
  'Power BI',
  'HANA Views',
  'DAX',
  'Power Automate',
  'BEx Analyzer',
  'BI Portal',
  'APDs',
  'QlikView',
  'S/4HANA',
  'Rebates Management',
  'Settlement Management',
  'PLM',
  'PM',
  'OTC',
  'CDS Views',
  'SDA',
  'Demantra',
  'ODP Framework',
  'ServiceNow',
  'JIRA',
  'Remedy',
  'Heat',
  'SAP BO Security',
  'Broadcasting',
  'SAP BOBJ WebI',
  'HANA Remediation',
  'Ticketing Tools'],
 'certifications': [],
 'key_responsibilities': ['Solution architect in analytics practice',
  'Solutioned webi to powerbi conversion project',
  'Created dataflows, datasets, reports, paginated reports',
  'Architected Power BI dashboard design',
  'Worked on BO 4.3 upgrade project',
  'Worked on BW/4HANA 1.0 green field implementation',
  'Worked on BW/4HANA 2.0 upgrade projects',
  'Worked extensively on SAP BI in various domains',
  'Worked with ABAP CD

In [ ]:
docx_path

'/content/resume_db/Candidate 105.docx'

**Start Weaviate Collection and Data Preparation**

In [ ]:
! pip install weaviate-client==4.16.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.5/597.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 26.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.0 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.0 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33

In [ ]:
WEAVIATE_URL = "eq9xrs3jtoep5ksrao86bw.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY = "cGxDeERtK0ZUUkQ5Wkt1Z19rWDJlWHNxZ0dzNnlmbFZSV2QvS2k3TER6ZDV4NEVMb2U3dTd3dGRteStZPV92MjAw"
OPENAI_API_KEY = ""
CollectionName = "resume"

# Create Weaviate Client

In [ ]:
import weaviate
import os

headers = {
    "X-Openai-Api-Key": OPENAI_API_KEY
}

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,  # Replace with your WCD URL
    auth_credentials=WEAVIATE_API_KEY, # Replace with your WCD key
    headers=headers,
)

# Create Collection

In [ ]:
from weaviate.classes.config import Property, DataType, Configure

if not client.collections.exists(CollectionName):
        client.collections.create(
            name=CollectionName,
            properties=[
                Property(name="resume_link", data_type=DataType.TEXT),
                Property(name="key_responsibilities", data_type=DataType.TEXT_ARRAY),
                Property(name="certifications", data_type=DataType.TEXT_ARRAY),
                Property(name="skills", data_type=DataType.TEXT_ARRAY),
                Property(name="year_of_experince", data_type=DataType.NUMBER),
                Property(name="resume_chunk", data_type=DataType.TEXT),
            ],
            vector_config=[
                Configure.Vectors.text2vec_openai(
                    name="key_responsibilities",
                    source_properties=["key_responsibilities"],
                    dimensions=512,
                    model="text-embedding-3-small",
                ),
                Configure.Vectors.text2vec_openai(
                    name="resume",
                    source_properties=["resume_chunk"],
                    dimensions=512,
                    model="text-embedding-3-small",
                ),
                  Configure.Vectors.text2vec_openai(
                    name="skills",
                    source_properties=["skills","certifications"],
                    dimensions=512,
                    model="text-embedding-3-small",
                )
            ],
        )
    # END_SOLUTION
else:
        raise RuntimeError(
            "Collection 'RESUME' already exists! "
            "If you like to re-build the collection, create and run a separate script to delete the existing collection. "
            "\n\nTo delete a collection, run: client.collections.delete(<collection_name>)"
        )

**Process each chunks and transform it into weaviate object and save to database**

In [ ]:
resume_collection = client.collections.use(CollectionName)

In [ ]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

# Enter context manager
with resume_collection.batch.fixed_size(batch_size=200) as batch:

    for chunk in tqdm(cleaned_chunks):
        # Build the object payload
        resume_obj = {
            "resume_link": docx_path,
            "key_responsibilities": [item.lower() for item in llm_response_json['key_responsibilities']],
            "certifications":[item.lower() for item in llm_response_json['certifications']],
            "skills": [item.lower() for item in llm_response_json['skills']],
            "year_of_experince": llm_response_json['experience_years_total'],
            "resume_chunk": chunk,
        }

        # Add object to batch queue
        batch.add_object(
            properties=resume_obj,
            uuid=generate_uuid5(chunk)
        )
        # Batcher automatically sends batches

# Check for failed objects
if len(resume_collection.batch.failed_objects) > 0:
    print(f"Failed to import {len(resume_collection.batch.failed_objects)} objects")

client.close()

100%|██████████| 41/41 [00:00<00:00, 12852.50it/s]


In [ ]:
client.collections.delete(CollectionName)